In [ ]:
import os
import sys
from sqlalchemy import Column, ForeignKey, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import MetaData
from sqlalchemy import Table, select, func, and_, insert, delete, update, or_
import numpy as np

In [ ]:
sql_credentials = {
  "host": "aquabyte-prod.cfwlu7jbdcqj.eu-west-1.rds.amazonaws.com",
  "port": "5432",
  "user": "aquabyte_ro",
  "password": "bJndX#uJ7VNVDJQW8g",
  "database": "aquabyte_prod"
}

In [ ]:
sql_engine = create_engine(
    "postgresql://{}:{}@{}:{}/{}".format(sql_credentials["user"], sql_credentials["password"],
                                         sql_credentials["host"], sql_credentials["port"],
                                         sql_credentials["database"]))

In [ ]:
metadata = MetaData()
# step 1 - download crops + json
fish_crops = Table('lati_fish_detections', metadata, autoload=True, autoload_with=sql_engine)
lice_crops = Table('lati_fish_detections_lice_annotations', metadata, autoload=True,
                   autoload_with=sql_engine)

# inner join on fish crop id
query = select([fish_crops.c.image_key,
                lice_crops.c.is_skipped,
                lice_crops.c.is_blurry,
                lice_crops.c.is_too_dark,
                lice_crops.c.is_obstructed,
                lice_crops.c.is_bad_crop,
                ]) \
    .select_from(lice_crops.join(fish_crops, lice_crops.c.lati_fish_detections_id == fish_crops.c.id)) \
    .where(and_(fish_crops.c.site_id == 23,
#                 lice_crops.c.is_skipped == True,
                ))

In [ ]:
connection = sql_engine.connect()
q = connection.execute(query)
results = [r for r in q]

In [ ]:
results = [r for r in results if r[0].split("/")[3].split("-")[0] == "2019"]

In [ ]:
from collections import Counter

In [ ]:
Counter([r[0].split("/")[1] for r in results])

In [ ]:
imagedic = {}
for r in results:
    image_name = "_".join(r[0].split("/")[-1].split("_")[:4])
    if image_name not in imagedic:
        imagedic[image_name] = []
    imagedic[image_name].append(r)
print("Total number of frames: {}".format(len(list(imagedic.keys()))))

In [ ]:
ncrops = []
for (k, v) in imagedic.items():
    ncrops.append(len(v))
print("Average number of crops per frame: {}".format(np.mean(ncrops)))

In [ ]:
c = Counter([len(v) for (k,v) in imagedic.items()])
print(c)
total = 0
for (k,v) in c.items():
    if k == 1:
        continue
    total += v
print("total number of frames with at least two crops: {}".format(total))

In [ ]:
all_bad = 0
for (k, v) in imagedic.items():
    if len(v) == 1:
        continue
    skip_counter = 0
    for crop in v:
        if crop[1]:
            skip_counter += 1
    if len(v) == skip_counter and skip_counter>0:
        all_bad += 1

print("Percentage of frames where all the crops are bad: {}".format(all_bad*100/total))